<a href="https://colab.research.google.com/github/venu72561-chinnam/AI-ML-project/blob/main/fashion_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gradio scikit-learn matplotlib tensorflow -q

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [3]:
from sklearn.decomposition import PCA
from sklearn.neighbors import KDTree

In [4]:
from tensorflow.keras.datasets import fashion_mnist

In [5]:
import gradio as gr
from PIL import Image

In [6]:
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [7]:
X_train_flat = X_train.reshape(len(X_train), -1) / 255.0
X_test_flat = X_test.reshape(len(X_test), -1) / 255.0

In [8]:
pca = PCA(n_components=50, random_state=42)

In [9]:
X_train_pca = pca.fit_transform(X_train_flat)
X_test_pca = pca.transform(X_test_flat)

In [10]:
tree = KDTree(X_train_pca, leaf_size=40)

In [11]:
labels_map = [
    "T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
    "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"
]

In [12]:
dist, ind = tree.query(X_test_pca[:1000], k=1)  # test on 1k samples
y_pred = y_train[ind.flatten()]
acc = np.mean(y_pred == y_test[:1000])
print("Retrieval Accuracy (Top-1):", acc)

Retrieval Accuracy (Top-1): 0.831


In [13]:
def find_similar(img):
    img_gray = img.convert("L")   # ensure grayscale
    img_resized = img_gray.resize((28,28))
    img_flat = np.array(img_resized).flatten().reshape(1, -1) / 255.0
    img_pca = pca.transform(img_flat)

    dist, ind = tree.query(img_pca, k=5)
    neighbors = ind[0]

    results = []
    captions = []
    for i in neighbors:
        results.append(Image.fromarray(X_train[i]))
        captions.append(labels_map[y_train[i]])
    return results, {labels_map[y_train[i]]: 1.0 for i in neighbors}

In [14]:
with gr.Blocks() as demo:
    gr.Markdown("## Image Similarity Project")
    with gr.Row():
        inp = gr.Image(type="pil", label="Upload a clothing image")
        gallery = gr.Gallery(label="Most Similar Images (Top-5)", columns=5, height="auto")
    out_labels = gr.Label(num_top_classes=5, label="Predicted Similar Categories")

    inp.upload(find_similar, inputs=inp, outputs=[gallery, out_labels])

In [15]:
demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6b3fbcf1eaa77ebdac.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
